In [0]:
import pandas as pd

In [2]:
data = pd.read_csv("nyc_bikeshare.csv")
print("done")

done


In [3]:
print(data)

        tripduration          starttime  ... birthyear  gender
0               2059  06-01-13 00:00:44  ...       NaN     0.0
1               1521  06-01-13 00:01:22  ...    1983.0     1.0
2               2028  06-01-13 00:01:47  ...       NaN     0.0
3               1829  06-01-13 00:03:47  ...    1984.0     1.0
4                899  06-01-13 00:09:25  ...    1967.0     1.0
...              ...                ...  ...       ...     ...
421951          1285  07-31-13 16:50:25  ...       NaN     0.0
421952          1333  07-31-13 16:50:31  ...    1959.0     1.0
421953          1539  07-31-13 16:50:33  ...       NaN     0.0
421954          1304  07-31-13 16:50:34  ...    1971.0     1.0
421955           458  07-31-13 16:50:36  ...      19.0     NaN

[421956 rows x 9 columns]


In [0]:
key = pd.read_csv("nyc_bikeshare_key.csv")

In [0]:
data1 = data.join(key.set_index("station_id"), on="start_station_id", rsuffix="_start")
data1 = data1.join(key.set_index("station_id"), on="end_station_id", rsuffix="_end")

In [7]:
startGroup = data1.groupby(["start_station_id", "station_latitude", "station_longitude"]).count()
endGroup = data1.groupby(["end_station_id"]).count()
startGroup = startGroup.filter(items=["tripduration"])
endGroup = endGroup.filter(items=["starttime"])
startGroup = startGroup.join(endGroup, on="start_station_id").rename(columns = {"tripduration": "starts", "starttime": "ends"})
startGroup = startGroup.fillna(0)
print(startGroup)

                                                     starts  ends
start_station_id station_latitude station_longitude              
72.0             40.767272        -73.993929           1609  1537
79.0             40.719116        -74.006667           2548  2566
82.0             40.711174        -74.000165            554   520
83.0             40.683826        -73.976323            834  1008
116.0            40.741776        -74.001497           1734  1629
...                                                     ...   ...
3002.0           40.711512        -74.015756            716   816
3014.0           40.722852        -73.959114             57    56
3017.0           40.751483        -73.996764            251   248
3019.0           40.716633        -73.981933             99    98
3020.0           40.646607        -74.015970             58    54

[338 rows x 2 columns]


In [0]:
startGroup.to_csv("output/stationCountSmall.csv", index=True)

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
startGroup.to_csv("/content/drive/My Drive/WestCoastDataOpen/stationCountSmall.csv", index=True)

In [21]:
# start_time -> year, month, date, time
from collections import defaultdict

times = defaultdict(list)
for entry in data1["starttime"]:
    parts = entry.split(" ")
    date = parts[0].split("-")
    times["year"].append(date[0])
    times["month"].append(date[1])
    times["day"].append(date[2])
    times["time_of_day"].append(parts[1][:2])
times_df = pd.DataFrame(times)
times_df.head()

,year,month,day,time_of_day
0,06,01,13,00
1,06,01,13,00
2,06,01,13,00
3,06,01,13,00
4,06,01,13,00


In [23]:
data1 = data1.reset_index()
data1 = data1.join(times_df)

ValueError: ignored

In [37]:
firstStationUse = data1
firstStationUse = firstStationUse.groupby(["start_station_id"])["starttime"].first()
firstStationUse = pd.DataFrame(firstStationUse)
firstStationUse

,starttime
start_station_id,
72.0,06-01-13 08:14:11
79.0,06-01-13 07:29:36
82.0,06-01-13 01:21:54
83.0,06-01-13 13:00:16
116.0,06-01-13 01:57:32
...,...
3002.0,06-01-13 12:45:51
3014.0,06-01-13 18:36:36
3017.0,06-01-13 15:45:07


In [40]:
from datetime import datetime
def date_to_days(date):
  return date.split(" ")[0]
def daysFrom(date):
  startDate = datetime.strptime('6/1/2013', "%m/%d/%Y")
  endDate = datetime.strptime(date, "%m-%d-%y")
  return (endDate-startDate).days

2


In [42]:
firstStationUse["starttime"] = firstStationUse["starttime"].apply(date_to_days)
firstStationUse

,starttime
start_station_id,
72.0,06-01-13
79.0,06-01-13
82.0,06-01-13
83.0,06-01-13
116.0,06-01-13
...,...
3002.0,06-01-13
3014.0,06-01-13
3017.0,06-01-13


In [43]:
firstStationUse["daysSinceStart"] = firstStationUse["starttime"].apply(daysFrom)
firstStationUse

,starttime,daysSinceStart
start_station_id,,
72.0,06-01-13,0
79.0,06-01-13,0
82.0,06-01-13,0
83.0,06-01-13,0
116.0,06-01-13,0
...,...,...
3002.0,06-01-13,0
3014.0,06-01-13,0
3017.0,06-01-13,0
